In [2]:
# !pip install fastembed langchain langchain_community PyMuPDF chromadb

## Ollama form langchain

In [10]:

from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")

llm.invoke("Hola, quien eres?")

C:\Users\asusg\AppData\Local\Temp\ipykernel_19340\1440260240.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


'¡Hola! Me alegra conocerte. Soy un modelo de lenguaje artificial llamado Llama. Llama significa "Meta AI de modelo de lenguaje grande" y es una abreviatura de "Large Language Model". Soy un modelo de lenguaje entrenado para entender y generar texto, lo que me permite responder a preguntas y tener conversaciones con usuarios como tú.\n\nNo tengo una identidad personal en el sentido tradicional, pero estoy diseñado para ser útil y amigable. Mi objetivo es ayudarte con cualquier pregunta o tema que desees discutir. ¿En qué puedo ayudarte hoy?'

# RAG

### Load Document

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from pathlib import Path

data_folder = Path("data")

pdf_paths = list(data_folder.glob("*.pdf"))
data_pdf = []

print(pdf_paths)
for path in pdf_paths:
    loader = PyMuPDFLoader(path)
    data_pdf.extend(loader.load())  # Une todos los documentos

print(f"Se cargaron {len(data_pdf)} páginas de PDFs.")


[WindowsPath('data/EfectosDeLaSuplementacionConCreatinaMonohidratada.pdf'), WindowsPath('data/NutricionSuplementacionEHidratacionEnElAmbitoDep.pdf')]
Se cargaron 25 páginas de PDFs.


Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-11-14T20:01:34+01:00', 'source': 'data\\EfectosDeLaSuplementacionConCreatinaMonohidratada.pdf', 'file_path': 'data\\EfectosDeLaSuplementacionConCreatinaMonohidratada.pdf', 'total_pages': 8, 'format': 'PDF 1.7', 'title': '', 'author': 'Framcisco Ruiz Juan', 'subject': '', 'keywords': '', 'moddate': '2024-11-14T20:01:34+01:00', 'trapped': '', 'modDate': "D:20241114200134+01'00'", 'creationDate': "D:20241114200134+01'00'", 'page': 0}, page_content='2025, Retos, 62, 958-965 \n© Copyright: Federación Española de Asociaciones de Docentes de Educación Física (FEADEF) ISSN: Edición impresa: 1579-1726. Edición Web: 1988-2041 (https://recyt.fecyt.es/in-\ndex.php/retos/index) \n-958-                                                                                                                                                                                                              R

In [4]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
docs = text_splitter.split_documents(data_pdf)

In [5]:

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\ESTUDIOS\UDIT\TFM\RAG_implementation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\ESTUDIOS\UDIT\TFM\RAG_implementation\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asusg\AppData\Local\Temp\fastembed_cache\models--qdrant--all-MiniLM-L6-v2-onnx. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either n

In [6]:
from langchain_community.vectorstores import Chroma

vs = Chroma.from_documents(
    documents=docs,
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="stanford_report_data"
)

In [7]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="chroma_db_dir",
                     collection_name="stanford_report_data")
retriever=vectorstore.as_retriever(search_kwargs={'k': 5})

C:\Users\asusg\AppData\Local\Temp\ipykernel_19340\2007608580.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embed_model,


In [8]:

from langchain.prompts import PromptTemplate

custom_prompt_template = """Usa la siguiente información para responder a la pregunta del usuario.
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta útil a continuación y nada más y responde siempre en español
Respuesta útil:
"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

In [11]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})

In [14]:
response = qa({"query": "¿Que suplementación es más importante para la mejora muscular?¿Por qué?"})
print(response['result'])

La suplementación con creatina monohidrata (CrM) es más importante para la mejora muscular que otros suplementos, ya que ha demostrado ser efectiva en mejorar la composición muscular, fuerza muscular y potencia muscular en adultos jóvenes entrenados. La ingesta de CrM puede ayudar a aumentar la cantidad de fosfocreatina en el cuerpo, lo que permite a los músculos realizar más contracciones con mayor intensidad y duración, lo cual es beneficioso para mejorar el rendimiento físico. Además, la suplementación con CrM no tiene efectos adversos significativos y se puede consumir de manera segura y eficaz.
